# Interactive Plotting for API Results
Interactive maps are a great way to present geospatial data, allowing people to zoom, pan, and click on results.  [Folium](https://python-visualization.github.io/folium/index.html) is a python library for making interactive maps built on top of a JavaScript mapping library called Leaflet. The results of queries made using the `osdatahub` are returned in GeoJSON format, which is an ideal input for interactive mapping libraries with JavaScript underpinnings, and the OS Maps API can supply tiles for basemaps giving that all important context to your data.

In [ ]:
from datetime import datetime
from osdatahub import Extent
from osdatahub import FeaturesAPI 
import os
os.environ['PROJ_NETWORK'] = 'OFF'

## Polygons with Popups
The example uses Zoomstack Surface Water to show how to plot the API results onto an interactive map and provide additional information about the data to users through popups.

### Get Data from the API

In [ ]:
# Choose data product
product = 'zoomstack_surface_water'

# Define query extent. Folium maps use coordinates in EPSG:4326,
# the extent is specified in degrees longitude and latitude
W, S = (-1.328, 50.979)
E, N = (-1.322, 50.9815)
crs = "EPSG:4326"
extent = Extent.from_bbox((W, S, E, N), crs=crs)

# Query API
key = os.environ.get("OS_API_KEY")
surface_water = FeaturesAPI(key, product, extent)
surface_water_results = surface_water.query(limit=100000)

### Define the Map
The first step in creating a folium map is to create a map object. This defines the location in view when the map first loads. Folium maps always use coordinates in the WGS84 coordinate reference system, so the starting location must be specified in degrees latitude and longitude - note the order.

In [ ]:
import folium

# Careful specifying MAP_CENTRE - is always [latitude, longitude] when
# elsewhere you might be using [longitude, latitude].
MAP_CENTRE = [50.98, -1.325]

# m is the folium map instance all other layers will be added to.
m = folium.Map(location=MAP_CENTRE,
               zoom_start=16,
               tiles=None)

### Add a Basemap
This defines the base map. You can access base map tiles via the OS Maps API. By adding the Maps API to the same project as the Features API, you can use the same key for both. To find out more about API keys, check out the "Setting up an API key" example. Folium requires you to provide attribution for the tiles, which will be displayed with the tiles.

In [ ]:
LAYER = "Outdoor_3857"
TILES = f"https://api.os.uk/maps/raster/v1/zxy/{LAYER}/{{z}}/{{x}}/{{y}}.png?key={key}"
ATTR = f"Contains OS data © Crown copyright and database right {datetime.now().year}"

tiles = folium.TileLayer(tiles=TILES,
                         attr=ATTR,
                         max_zoom=16,
                         name='OS Maps Outdoor',
                         control=False).add_to(m)

### Add the Data
Add the API results to the map with a popup showing the area of each polygon.

In [ ]:
# The popup text can be populated directly from the properties of the water polygons
# by specifying the name of the field and an optional display name.
popup = folium.GeoJsonPopup(fields=['SHAPE_Area'], aliases=["Area (m2)"])
water_layer = folium.GeoJson(surface_water_results,
                             name="Zoomstack Surface Water",
                             popup=popup).add_to(m)

### Add Bounding Box

In [ ]:
from shapely.geometry import mapping

# Convert the extent into GeoJSON using shapely's mapping() function
bbox_geojson = mapping(extent.polygon)

# Specify the styling for plotting the bbox.
bbox_styling = {'fillColor': 'none',
                'color': 'black',
                'lineOpacity': 0.9,
                'weight': 1.2,
                'dashArray': 10}

# Add the bounding box to the map.
folium.GeoJson(bbox_geojson,
               style_function=lambda x: bbox_styling,
               name="Bounding Box").add_to(m)

# Add ability to toggle layers
folium.LayerControl().add_to(m)

# View the map
m

# Heatmaps
This example demonstrates how to plot the density of different address types as heatmaps. Folium heatmaps show brighter colors where there are more points in an area. This is a good way to look at what areas of a town are residential vs retail, for example. In this example, you can clearly see where the local high street is and where housing tends to be terraced vs semi-detached.

### Get Data From the API
With the option to define the extent of a query using an ONS code, this query finds the location of all addresses within a particular Lower Super Output Area. The resulting points are split into seperate lists depending on the classification of the address.

In [ ]:
from osdatahub import PlacesAPI
from collections import defaultdict

# Define the extent as Portswood LSOA
portswood = "E01017235"
extent = Extent.from_ons_code(portswood)

# Query the API
places = PlacesAPI(key)
addresses = places.query(extent=extent, output_crs=crs, limit=10000)

# Split the results by classification. This makes a dictionary with the keys being
# the classifications and the values a list of all addresses with that classification.
classifications = defaultdict(list)
for feature in addresses["features"]:
    address_classification = feature["properties"]['CLASSIFICATION_CODE_DESCRIPTION']
    # Notice the [::-1] at the end. The reverses the coordinates because
    # folium uses (lat, lon), rather than (x, y)
    classifications[address_classification].append(feature["geometry"]["coordinates"][::-1])

### Define the Map

In [ ]:
from folium.plugins import HeatMap

# Define the map
m = folium.Map(location=[50.924, -1.391],
               zoom_start=16,
               tiles=None)
# Add the basemap tiles 
folium.TileLayer(tiles=TILES,
                 attr=ATTR,
                 max_zoom=16,
                 name='OS Maps Light',
                 control=False).add_to(m)

# Add the extent
extent_geojson = mapping(extent.polygon)
extent_layer = folium.GeoJson(extent_geojson,
                              style_function=lambda x: bbox_styling,
                              name=f"{portswood} Ward Boundary", control=False).add_to(m)

### Plot Heatmaps by Address Classification
Adding each classification seperately let the users select which classification they are interested in from the layers panel that appears in the top right of the map.

In [ ]:
# Loop through the classifications, adding each one as a layer
for classification, points in classifications.items():
    HeatMap(points, radius=20, blur=25, name=classification, overlay=False).add_to(m)

# Add the ability to toggle between the different classification heatmaps.
folium.LayerControl().add_to(m)
m